Гопиенко Александр
КИ22-17/2Б
Практическая работа №5 по машинному обучению 
Задание №1

In [82]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.impute import KNNImputer
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, StackingClassifier
from sklearn.ensemble import BaggingRegressor, AdaBoostRegressor, StackingRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.linear_model import LogisticRegression, LinearRegression

Выполним обработку датасета telecom_churn для классификации
Начнём с кодирования категориальных признаков

In [83]:
df_clf = pd.read_csv("telecom_churn.csv")
df_clf['Churn'] = df_clf['Churn'].astype(int)
encoder = OneHotEncoder(sparse_output=False, drop='first')
categorical_features = df_clf.select_dtypes(include=['object']).columns
encoded_features = encoder.fit_transform(df_clf[categorical_features])
df_encoded = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_features))
df_clf = pd.concat([df_clf, df_encoded], axis=1)
df_clf.drop(categorical_features, axis=1, inplace=True)
df_clf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 69 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Account length          3333 non-null   int64  
 1   Area code               3333 non-null   int64  
 2   Number vmail messages   3333 non-null   int64  
 3   Total day minutes       3333 non-null   float64
 4   Total day calls         3333 non-null   int64  
 5   Total day charge        3333 non-null   float64
 6   Total eve minutes       3333 non-null   float64
 7   Total eve calls         3333 non-null   int64  
 8   Total eve charge        3333 non-null   float64
 9   Total night minutes     3333 non-null   float64
 10  Total night calls       3333 non-null   int64  
 11  Total night charge      3333 non-null   float64
 12  Total intl minutes      3333 non-null   float64
 13  Total intl calls        3333 non-null   int64  
 14  Total intl charge       3333 non-null   

Избавимся от выбросов в датасете telecom_churn

In [84]:
def detect_outliers(column):
    Q1 = column.quantile(0.25)
    Q3 = column.quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = (column < lower_bound) | (column > upper_bound)
    return outliers

column_to_check = df_clf.iloc[:, :16]
df_res = df_clf.iloc[:, 16:]
Q1 = column_to_check.quantile(0.25)
Q3 = column_to_check.quantile(0.75)

# Рассчитываем межквартильный размах (IQR)
IQR = Q3 - Q1

# Определяем границы для определения выбросов
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR


# Фильтруем строки, удаляя выбросы
column_to_check = column_to_check[(column_to_check >= lower_bound) & (column_to_check <= upper_bound)]
df_clf = pd.concat([column_to_check, df_res], axis=1)

for column in df_clf.columns:
    if pd.api.types.is_numeric_dtype(df_clf[column]):
        outliers = detect_outliers(df_clf[column])
        print(f"Количество выбросов в столбце '{column}': {outliers.sum()} ({round((outliers.sum() /df_clf[column].size), 2)})")
        

Количество выбросов в столбце 'Account length': 0 (0.0)
Количество выбросов в столбце 'Area code': 0 (0.0)
Количество выбросов в столбце 'Number vmail messages': 0 (0.0)
Количество выбросов в столбце 'Total day minutes': 2 (0.0)
Количество выбросов в столбце 'Total day calls': 5 (0.0)
Количество выбросов в столбце 'Total day charge': 2 (0.0)
Количество выбросов в столбце 'Total eve minutes': 1 (0.0)
Количество выбросов в столбце 'Total eve calls': 0 (0.0)
Количество выбросов в столбце 'Total eve charge': 1 (0.0)
Количество выбросов в столбце 'Total night minutes': 2 (0.0)
Количество выбросов в столбце 'Total night calls': 0 (0.0)
Количество выбросов в столбце 'Total night charge': 2 (0.0)
Количество выбросов в столбце 'Total intl minutes': 0 (0.0)
Количество выбросов в столбце 'Total intl calls': 0 (0.0)
Количество выбросов в столбце 'Total intl charge': 0 (0.0)
Количество выбросов в столбце 'Customer service calls': 0 (0.0)
Количество выбросов в столбце 'Churn': 483 (0.14)
Количество 

Избавимся от пропусков

In [85]:
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(df_clf)
df_clf = pd.DataFrame(normalized_data, columns=df_clf.columns)

imputer = KNNImputer()
df_filled = imputer.fit_transform(df_clf)
df_clf = pd.DataFrame(df_filled, columns=df_clf.columns)
df_clf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 69 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Account length          3333 non-null   float64
 1   Area code               3333 non-null   float64
 2   Number vmail messages   3333 non-null   float64
 3   Total day minutes       3333 non-null   float64
 4   Total day calls         3333 non-null   float64
 5   Total day charge        3333 non-null   float64
 6   Total eve minutes       3333 non-null   float64
 7   Total eve calls         3333 non-null   float64
 8   Total eve charge        3333 non-null   float64
 9   Total night minutes     3333 non-null   float64
 10  Total night calls       3333 non-null   float64
 11  Total night charge      3333 non-null   float64
 12  Total intl minutes      3333 non-null   float64
 13  Total intl calls        3333 non-null   float64
 14  Total intl charge       3333 non-null   

Выполним стандартизацию данных

In [86]:
columns_except_one = [col for col in df_clf.columns if col != 'Churn']
X = df_clf[columns_except_one]
y = df_clf['Churn']
scaler = StandardScaler()
scaled_data = scaler.fit_transform(X)
df = pd.DataFrame(scaled_data, columns=X.columns)
df = pd.concat([df, y], axis=1)
columns_except_one = [col for col in df.columns if col != 'Churn']
X = df[columns_except_one]
y = df['Churn']

Построим ML модели классификации

In [87]:
X = X.to_numpy()
y = y.to_numpy()

# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Одиночное дерево решений
decision_tree = DecisionTreeClassifier(random_state=42)
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid_search_tree = GridSearchCV(decision_tree, param_grid, cv=5, scoring='accuracy')
grid_search_tree.fit(X_train, y_train)
best_tree = grid_search_tree.best_estimator_
best_tree.fit(X_train, y_train)
y_pred_tree = best_tree.predict(X_test)

# Бэгинг классификация
bagging = BaggingClassifier(estimator=DecisionTreeClassifier(random_state=42), random_state=42)
param_grid = {'n_estimators': [10, 50, 100]}
grid_search_bagging = GridSearchCV(bagging, param_grid, cv=5)
grid_search_bagging.fit(X_train, y_train)
best_bagging = grid_search_bagging.best_estimator_
best_bagging.fit(X_train, y_train)
y_pred_bagging = best_bagging.predict(X_test)

# Бустинг классификация
adaboost = AdaBoostClassifier(estimator=DecisionTreeClassifier(random_state=42), random_state=42)
param_grid = {'n_estimators': [10, 50, 100]}
grid_search_adaboost = GridSearchCV(adaboost, param_grid, cv=5)
grid_search_adaboost.fit(X_train, y_train)
best_adaboost = grid_search_adaboost.best_estimator_
best_adaboost.fit(X_train, y_train)
y_pred_adaboost = best_adaboost.predict(X_test)

# Стекинг классификация
estimators = [('decision_tree', DecisionTreeClassifier(random_state=42)),
              ('bagging', BaggingClassifier(estimator=DecisionTreeClassifier(random_state=42), random_state=42)),
              ('adaboost', AdaBoostClassifier(estimator=DecisionTreeClassifier(random_state=42), random_state=42))]
stacking = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
param_grid = {'final_estimator__C': [0.1, 1, 10]}
grid_search_stacking = GridSearchCV(stacking, param_grid, cv=5)
grid_search_stacking.fit(X_train, y_train)
best_stacking = grid_search_stacking.best_estimator_
best_stacking.fit(X_train, y_train)
y_pred_stacking = best_stacking.predict(X_test)

accuracy_dt = accuracy_score(y_test, y_pred_tree)
accuracy_bagging = accuracy_score(y_test, y_pred_bagging)
accuracy_adaboost = accuracy_score(y_test, y_pred_adaboost)
accuracy_stacking = accuracy_score(y_test, y_pred_stacking)

print("Accuracy of Decision Tree:", accuracy_dt)
print("Accuracy of Bagging:", accuracy_bagging)
print("Accuracy of AdaBoost:", accuracy_adaboost)
print("Accuracy of Stacking:", accuracy_stacking)

Accuracy of Decision Tree: 0.9115442278860569
Accuracy of Bagging: 0.9400299850074962
Accuracy of AdaBoost: 0.8845577211394303
Accuracy of Stacking: 0.9295352323838081


Значения Accuracy из третьей работы:
KNN Accuracy: 0.8470764617691154
Logistic Regression Accuracy: 0.8545727136431784
SVM Accuracy: 0.8575712143928036

Эти значения меньше, чем у модели дерева и ассамблей.
Что говорит о более высокой точности моделей дерева и ассамблей

Обрежем модель дерева

In [88]:
tree_pruned = DecisionTreeClassifier(max_depth=3, random_state=42)
tree_pruned.fit(X_train, y_train)
y_pred_pruned = tree_pruned.predict(X_test)
accuracy_pruned = accuracy_score(y_test, y_pred_pruned)

# Вывод результатов
print("Accuracy of Pruned Decision Tree:", accuracy_pruned)

Accuracy of Pruned Decision Tree: 0.8995502248875562


Вывод:после обрезки у дерева упала точность, что может указывать на то, что обрезка привела к потере некоторых важных зависимостей в данных из-за уменьшения глубины дерева. Таким образом, можно доказать, что изменение глубины дерева влияет на результат.

Задание №2
Задачи регрессии

Используем датасет house_price

In [89]:
df_reg = pd.read_csv('house_price.csv')
df_reg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 808 entries, 0 to 807
Data columns (total 48 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         808 non-null    int64  
 1   Количество.комнат          808 non-null    int64  
 2   Студия                     808 non-null    int64  
 3   Общая.площадь              808 non-null    float64
 4   Этаж                       808 non-null    int64  
 5   Этажей.в.доме              808 non-null    int64  
 6   Парковка                   808 non-null    int64  
 7   Без.ремонта                808 non-null    int64  
 8   Дизайнерский               808 non-null    int64  
 9   Евроремонт                 808 non-null    int64  
 10  Косметический              808 non-null    int64  
 11  Балкон                     808 non-null    int64  
 12  Лоджия                     808 non-null    int64  
 13  Совмещенный.санузел        808 non-null    int64  

Избавимся от выбросов

In [90]:
def detect_outliers(column):
    Q1 = column.quantile(0.25)
    Q3 = column.quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = (column < lower_bound) | (column > upper_bound)
    return outliers

columns_check_1 = df_reg[['Общая.площадь', 'Этаж', 'Этажей.в.доме']]
columns_check_2 = df_reg.loc[:, 'Школа.1000':'Аптека.1000']
columns_check = pd.concat([columns_check_1, columns_check_2], axis=1)
Q1 = df_reg.quantile(0.25)
Q3 = df_reg.quantile(0.75)

# Рассчитываем межквартильный размах (IQR)
IQR = Q3 - Q1

# Определяем границы для определения выбросов
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR


# Фильтруем строки, удаляя выбросы
columns_check = columns_check[(columns_check >= lower_bound) & (columns_check <= upper_bound)]
other_columns = df_reg[[col for col in df_reg.columns if col not in columns_check]]
df_reg = pd.concat([columns_check, other_columns], axis=1)

for column in df_reg.columns:
    if pd.api.types.is_numeric_dtype(df_reg[column]):
        outliers = detect_outliers(df_reg[column])
        print(f"Количество выбросов в столбце '{column}': {outliers.sum()} ({round((outliers.sum() /df_reg[column].size), 2)})")
        

Количество выбросов в столбце 'Общая.площадь': 0 (0.0)
Количество выбросов в столбце 'Этаж': 0 (0.0)
Количество выбросов в столбце 'Этажей.в.доме': 0 (0.0)
Количество выбросов в столбце 'Школа.1000': 0 (0.0)
Количество выбросов в столбце 'ВУЗ.1000': 40 (0.05)
Количество выбросов в столбце 'ТЦ.1000': 0 (0.0)
Количество выбросов в столбце 'Стоматология.1000': 21 (0.03)
Количество выбросов в столбце 'Почта.1000': 0 (0.0)
Количество выбросов в столбце 'Поликлиника.1000': 0 (0.0)
Количество выбросов в столбце 'Парк.1000': 0 (0.0)
Количество выбросов в столбце 'Остановка.1000': 0 (0.0)
Количество выбросов в столбце 'Одежда..1000': 34 (0.04)
Количество выбросов в столбце 'Супер3кет.1000': 0 (0.0)
Количество выбросов в столбце 'Кинотеатр.1000': 0 (0.0)
Количество выбросов в столбце 'Кафе.1000': 26 (0.03)
Количество выбросов в столбце 'АЗС.1000': 0 (0.0)
Количество выбросов в столбце 'Детский.сад.1000': 15 (0.02)
Количество выбросов в столбце 'Бар.1000': 41 (0.05)
Количество выбросов в столбце 

C:\Users\sagop\AppData\Local\Temp\ipykernel_9496\1129291462.py:27: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version. Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  columns_check = columns_check[(columns_check >= lower_bound) & (columns_check <= upper_bound)]


Избавимся от пропусков

In [91]:
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(df_reg)
df_reg = pd.DataFrame(normalized_data, columns=df_reg.columns)


imputer = KNNImputer()
df_filled = imputer.fit_transform(df_reg)
df_reg = pd.DataFrame(df_filled, columns=df_reg.columns)
df_reg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 808 entries, 0 to 807
Data columns (total 48 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Общая.площадь              808 non-null    float64
 1   Этаж                       808 non-null    float64
 2   Этажей.в.доме              808 non-null    float64
 3   Школа.1000                 808 non-null    float64
 4   ВУЗ.1000                   808 non-null    float64
 5   ТЦ.1000                    808 non-null    float64
 6   Стоматология.1000          808 non-null    float64
 7   Почта.1000                 808 non-null    float64
 8   Поликлиника.1000           808 non-null    float64
 9   Парк.1000                  808 non-null    float64
 10  Остановка.1000             808 non-null    float64
 11  Одежда..1000               808 non-null    float64
 12  Супер3кет.1000             808 non-null    float64
 13  Кинотеатр.1000             808 non-null    float64

Обработаем коррелирующие признаки

In [92]:
correlation_matrix = df_reg.corr().abs()
threshold = 0.8
mask = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(np.bool_))
collinear_features = [column for column in mask.columns if any(mask[column] > threshold)]
df_reg = df_reg.drop(columns=collinear_features[1::2])

Выполним масштабирование признаков

In [93]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_reg)
df_reg = pd.DataFrame(scaled_data, columns=df_reg.columns)
y = df_reg['Цена']
columns_except_one = [col for col in df_reg.columns if col != 'Цена']
X = df_reg[columns_except_one]

Обучим ML модели регрессии

In [94]:
X = X.to_numpy()
y = y.to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Регрессия единичного дерева
tree_regressor = DecisionTreeRegressor(random_state=42)

param_grid = {
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid_search = GridSearchCV(estimator=tree_regressor, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)
best_tree_regressor = grid_search.best_estimator_
y_pred = best_tree_regressor.predict(X_test)
mse_test = mean_squared_error(y_test, y_pred)
print("MSE регрессии единичного дерева:", mse_test)

# Bagging регрессия
bagging_regressor = BaggingRegressor(estimator=DecisionTreeRegressor(random_state=42), random_state=42)

param_grid = {
    'n_estimators': [10, 50, 100],
    'max_samples': [0.5, 0.7, 1.0],
    'max_features': [0.5, 0.7, 1.0],
}

grid_search = GridSearchCV(estimator=bagging_regressor, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)
best_bagging_regressor = grid_search.best_estimator_
y_pred = best_bagging_regressor.predict(X_test)
mse_test = mean_squared_error(y_test, y_pred)
print("MSE Bagging регрессии:", mse_test)

# AdaBoost регрессия
adaboost_regressor = AdaBoostRegressor(estimator=DecisionTreeRegressor(random_state=42), random_state=42)

param_grid = {
    'n_estimators': [50, 100, 200]
}

grid_search = GridSearchCV(estimator=adaboost_regressor, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)
best_adaboost_regressor = grid_search.best_estimator_
y_pred = best_adaboost_regressor.predict(X_test)
mse_test = mean_squared_error(y_test, y_pred)
print("MSE AdaBoost регрессии:", mse_test)

# Stacking регрессия
base_model = [
    ('random_forest', RandomForestRegressor(random_state=42)),
    ('decision_tree', DecisionTreeRegressor(random_state=42))
]
meta_model = LinearRegression()
stacking_regressor = StackingRegressor(estimators=base_model, final_estimator=meta_model)

param_grid = {
    'random_forest__n_estimators': [50, 100]
}
grid_search = GridSearchCV(estimator=stacking_regressor, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)
best_stacking_regressor = grid_search.best_estimator_
y_pred = best_stacking_regressor.predict(X_test)
mse_test = mean_squared_error(y_test, y_pred)
print("MSE Stacking регрессии:", mse_test)

MSE регрессии единичного дерева: 0.1851630995674738
MSE Bagging регрессии: 0.17232660107249712
MSE AdaBoost регрессии: 0.17845796587277907
MSE Stacking регрессии: 0.17682533227425018


Сравним результаты единичного дерева и ассамблей со значениями регрессий из четвёртой работы

Значение среднеквадратичной ошибки (MSE) для линейной регрессии: 0.18084396740561526
Значение среднеквадратичной ошибки (MSE) для лассо регрессии: 0.2077510861887075
Значение среднеквадратичной ошибки (MSE) для гребневой регрессии: 0.17798457050904343
Значение среднеквадратичной ошибки (MSE) для SVR регрессии: 0.17074521360091618
Среднеквадратичная ошибка (MSE) для полиномиальной регрессии: 60.35375280261971

Результаты единичного дерева и ассамблей лучше моделей лассо и полиномиальных регрессий
Самая лучшая модель Bagging регрессии уступает в качестве SVR регрессии

Обрежем модель дерева

In [95]:
tree_model_pruned = DecisionTreeRegressor(max_depth=3, random_state=42)
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
pruned_scores = cross_val_score(tree_model_pruned, X_train, y_train, cv=kfold, scoring='neg_mean_squared_error')
mean_pruned_score = -pruned_scores.mean()
print("MSE обрезанного дерева:", mean_pruned_score)

MSE обрезанного дерева: 0.22315644368293


Вывод:после обрезки у дерева упала точность, что может указывать на то, что обрезка привела к потере некоторых важных зависимостей в данных из-за уменьшения глубины дерева. Таким образом, можно доказать, что изменение глубины дерева влияет на результат.

Общий вывод: изучены модели классификации и регрессии единичного дерева и ассамблей